In [2]:
import pandas as pd
import numpy as np

In [3]:
import os

print(os.getcwd())

/Users/buldiga/Documents/ml/book-recsys/notebooks


In [33]:
df = pd.read_csv('../large_files/interactions.csv')
users = pd.read_csv('../large_files/users.csv')

In [34]:
df["start_date"] = pd.to_datetime(df["start_date"])
min_date = df["start_date"].max().normalize() - pd.DateOffset(days=30)

In [29]:
rec = (
            df.loc[df["start_date"] > min_date, "item_id"]
            .value_counts()
            .head(100)
            .index.values)

In [30]:
rec

array([283713, 276903, 184549, 168963, 357309, 143175, 385281,  80003,
        55466, 287060, 352049, 187325, 267817, 112869, 242176,  79499,
        50718, 325439,  33801, 127019, 323949, 321351, 390771,  93751,
        56877, 360740, 262464, 334106, 290266, 374648,   5408,  21313,
        51423, 190198,  40953, 302067, 160349, 131612, 194169, 231623,
       273112, 117325,   3465, 229030,  91236, 246948, 112796, 352701,
       256084, 201080, 364570, 178529, 354482, 232060, 344047, 350854,
        11316,  10224, 389882, 267878, 224853, 318926, 220922, 313220,
       247682, 229111, 126630, 305061, 249083, 122140, 143526, 267291,
       140100, 379918,  34223, 109091, 173173,  69198, 202054,  85489,
       292395, 310921,  50124, 281368, 268571, 229939,  76747, 280876,
       279488, 137878, 344432, 207599,  67966, 166662, 202142,  55495,
       337820, 353554, 143687,  93398])

In [35]:
merged = df.merge(users, on='user_id')

In [9]:
df[df['progress'] < 50].shape, df[df['progress'] > 50].shape

((826154, 5), (727426, 5))

In [10]:
submission = pd.read_csv("../large_files/sample_submission.csv")
pred_pop = pd.DataFrame({"user_id": submission["Id"].unique()})

In [11]:
user_ids_pred = set(list(pred_pop['user_id']))

In [12]:
user_ids = set(list(users['user_id']))

In [13]:
len(user_ids_pred.difference(user_ids)), len(user_ids_pred.intersection(user_ids))

(547, 2527)

In [14]:
usr = pred_pop.merge(users, how='left', on='user_id').fillna(1.0)

In [15]:
for index, row in usr.iterrows():
    print(row)
    if row['sex'] == 0.0:
        print('woman')
    break

user_id    10001
age        25_34
sex          0.0
Name: 0, dtype: object
woman


In [19]:
users.groupby('age').count()

,user_id,sex
age,,
18_24,28413,28097
25_34,36701,36449
35_44,31373,31185
45_54,22636,22293
55_64,11803,11488
65_inf,6318,6118


In [47]:
groups = merged.loc[merged["start_date"] > min_date, ["item_id", 'age']].fillna('25_34').groupby('age')

In [59]:
rec = {}
for name, group in groups:
    rec[name] = group['item_id'].value_counts().head(10).index.values
print(rec)

{'18_24': array([ 80003, 283713, 184549, 385281, 287060, 267817, 276903, 168963,
       112869,  56877]), '25_34': array([ 80003, 357309, 385281, 184549, 276903,  56877, 283713, 287060,
       168963, 262464]), '35_44': array([184549, 283713, 357309, 385281, 168963,  80003, 242176, 276903,
       287060,  56877]), '45_54': array([283713, 184549, 143175, 168963, 276903,  55466, 357309, 242176,
       321351,  50718]), '55_64': array([283713, 184549, 143175,  55466, 168963, 276903, 160349,  51423,
       323949,  79499]), '65_inf': array([283713, 184549, 143175,  55466, 160349,  51423, 374648, 276903,
       178529, 168963])}
